In [2]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline
import numpy as np
import sympy
import scipy.sparse.linalg
from math import pi

import setup_grids
from core.constit import second_order_tensor
from core.bc import bc
from fvdiscr import mpfa, fvutils
from viz import cell_data

In [90]:
# Homogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 5
grid_type = 'cart'
pert = 0.5
### End of parameter definitions

# Permeability tensor, scalar for simplicity
perm = 1

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = second_order_tensor.SecondOrderTensor(2, perm * np.ones(g.num_cells))

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.get_boundary_faces()
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux = mpfa.mpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros(g.num_faces)
        u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes + div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = scipy.sparse.linalg.spsolve(a, b)
        flux_num = flux * u_num - bound_flux * u_bound

        # Exact solution
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals * du_ex_faces, axis=0)
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print(u)
    print(f)


[0.0073686732581183549, 0.002061414152465271, 0.00052069850436349931, 0.00012689494678641146, 3.3174376429587348e-05]
[0.018364051591445909, 0.0069607346238992522, 0.0018965063877403581, 0.00076139949877771179, 0.00034071679478240468]
[0.002612929254681227, 0.00060836749272112438, 0.00016721778412008068, 3.895167304795528e-05, 9.5183746492585204e-06]
[0.011939673878239845, 0.005987844046830784, 0.0019404259450681617, 0.00080962523721631757, 0.00037383233647144185]
 

In [3]:
# Heterogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should stay unaltered
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 1
grid_typ = 'cart'
pert = 0.5

kappa_list = [1e-6, 1, 1e6]

### End of parameter definitions

# Permeability tensor, scalar for simplicity
perm = 1

def chi(x, y):
    return np.logical_and(np.greater(x, 0.5), np.greater(y, 0.5))

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(2 * x * pi) * sympy.sin(2 * y * pi) 
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    u_err = np.zeros((num_refs, len(kappa_list)))
    flux_err = np.copy(u_err)
    
    for iter1, g in enumerate(setup_grids.grid_sequence_fixed_lines(basedim, num_refs, grid_type, pert, subdom_func=chi)):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        for iter2, kappa in enumerate(kappa_list):
            char_func_cells = chi(g.cell_centers[0], g.cell_centers[1]) * 1.
            perm_vec = (1 - char_func_cells) + kappa * char_func_cells
            perm = second_order_tensor.SecondOrderTensor(2, perm_vec)

            bound_faces = g.get_boundary_faces()
            bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
            flux, bound_flux = mpfa.mpfa(g, perm, bound_cond, inverter='python')

            xc = g.cell_centers
            xf = g.face_centers
            char_func_bound = chi(xf[0, bound_faces], xf[1, bound_faces]) * 1
            
            u_bound = np.zeros(g.num_faces)
            u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces]) \
                                / ((1 - char_func_bound) + kappa * char_func_bound)

            div = fvutils.scalar_divergence(g)
            a = div * flux

            b = rhs_f(xc[0], xc[1]) * g.cell_volumes + div * bound_flux * u_bound

            u_num = scipy.sparse.linalg.spsolve(a, b)
            u_ex = u_f(xc[0], xc[1]) / ((1 - char_func_cells) + kappa * char_func_cells)
            
            flux_num = flux * u_num - bound_flux * u_bound
            du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
            flux_ex = -np.sum(g.face_normals * du_ex_faces, axis=0)
            flux_diff = flux_num - flux_ex

            u_err[iter1, iter2] = (np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                         np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
            flux_err[iter1, iter2] = (np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                            np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print(u)
    print(f)


[[ 0.28144397  0.3116494   0.28115807]]
[[ 0.18233422  0.18698603  0.20433553]]
[[ 0.0909746   0.18646187  0.14360106]]
[[ 0.1395195   0.1329257   0.13624701]]


In [78]:
print(dux_f(0.5, 0.7))

5.97566432948
